In [9]:
%pip install fastapi pydantic toml openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check, chat_with_azure
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat_with_azure(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        difficulty = dict["difficulty"]
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add round summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        if ("memories" not in config.metadata):
            config.metadata["memories"] = []
            
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "ending" in dict.keys():
        ending_btn = widgets.Button(description="END", layout=widgets.Layout(width="auto"))
        buttons.append(ending_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/love_cruise.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round =1
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

context: You make your way towards your suite, and you are immediately greeted by a spacious room with a balcony. As you set your luggage and items in place, the Captain, through the intercom, invites everyone to the top deck for food and drinks as the cruise departs. You...
actions:
1. Attend the party on the deck
2. Stay in your suite and turn on the TV
3. Go to the balcony and relax


I select option 3. TURN: 1 


context: As you step out onto the balcony, the cool sea breeze greets you, carrying with it the faint sounds of laughter and music from the deck party above. The sky is painted with hues of orange and pink as the sun begins to set, creating a serene atmosphere. As you lean over the railing to take in the view, you notice a girl standing on the balcony next to yours. She seems to be lost in thought, gazing out at the ocean.
actions:
1. Say hello and introduce yourself.
2. Enjoy the view in silence, respecting her privacy.
3. Ask her what she thinks about the view.

I select option 3. TURN: 2 


context: Turning towards her, you decide to break the ice with a simple question. "Beautiful view, isn't it? What do you think?" She looks over, a bit surprised but then smiles gently, nodding. "Yes, it's like a painting that changes every moment," she responds, her voice calm and collected. "I'm Mitsuha, by the way," she introduces herself, extending a hand in greeting. The two of you exchange names and pleasantries, finding common ground in your appreciation for the serene beauty of the sea.
actions:
1. Ask Mitsuha what brings her on this cruise.
2. Invite Mitsuha to join you at the party on the deck.
3. Share your reason for being on the cruise.

I select option 3. TURN: 3 


context: You share with Mitsuha that you're on the cruise to take a break from your routine and hopefully meet new people. "It's not often I get to step away from my programming job, so I figured this would be a perfect opportunity," you explain, watching her reaction. Mitsuha nods understandingly, "That sounds like a wonderful reason. It's important to take breaks and experience new things," she says, her tone warm and encouraging. The conversation flows smoothly, with both of you sharing laughs and stories.
actions:
1. Ask Mitsuha about her interests.
2. Suggest going to get some fresh air with a walk around the ship.
3. Offer to buy her a drink from the party on the deck.

I select option 1. TURN: 4 


context: Curious about Mitsuha, you ask about her interests. She lights up, sharing that she loves to read and game. "I find both to be perfect ways to escape and explore new worlds," she explains, her eyes sparkling with enthusiasm. "What about you? Do you enjoy reading or gaming?" she inquires, genuinely interested in your response. The conversation delves into favorite books and games, revealing a shared passion for storytelling and adventure.
actions:
1. Recommend a book or game to Mitsuha.
2. Invite Mitsuha to play a game together sometime.
3. Share a memorable adventure you've had in a game or book.

I select option 1. TURN: 5 


context: You recommend "The Night Circus" by Erin Morgenstern to Mitsuha, describing it as a magical and captivating read that you think she'd enjoy. "It's about a mysterious circus that only appears at night, and it's filled with enchantment and intrigue," you explain. Mitsuha listens intently, her interest piqued. "That sounds fascinating! I'll definitely give it a read. Thank you for the recommendation," she says with a smile. The conversation continues, and you both discover more shared interests.
summary: As the sun sets, you find yourself engaged in a delightful conversation with Mitsuha, a girl who loves to read and game. You learn about each other's interests and recommend a book to her, feeling a connection forming. [Mitsuha]